In [1]:
import pandas as pd

def keep_first_row(group):
    return group.iloc[0]

df = pd.read_csv("dataset/ner_dataset.csv",encoding="unicode_escape")

df['Sentence #'].ffill(inplace=True)

# df['sentence'] = df.groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
df['sentence'] = df.groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(str(word) for word in x))
df['tokens'] = df['sentence'].apply(lambda x: x.split())
df.drop("sentence",axis=1,inplace=True)

df['tags'] = df.groupby(['Sentence #'])['Tag'].transform(lambda x: ' '.join(str(tag) for tag in x))
df['ner_tags'] = df['tags'].apply(lambda x: x.split())
df.drop("tags",axis=1,inplace=True)

df.drop(["Word","POS","Tag"],axis=1,inplace=True)
df['Sentence #'] = df['Sentence #'].apply(lambda x: x.split(" ")[-1])
df.rename(columns={"Sentence #":"id"},inplace=True)

df = df.groupby('id').apply(keep_first_row)

In [10]:
df["id"] = df['id'].apply(lambda x: int(x))
df = df.set_index("id").sort_index()

In [12]:
df.reset_index()

,id,tokens,ner_tags
0,1,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,2,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,3,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo..."
3,4,"[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,5,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,..."
...,...,...,...
47954,47955,"[Indian, border, security, forces, are, accusi...","[B-gpe, O, O, O, O, O, O, B-gpe, O, O, O, O, O..."
47955,47956,"[Indian, officials, said, no, one, was, injure...","[B-gpe, O, O, O, O, O, O, O, B-tim, O, O, O, O..."
47956,47957,"[Two, more, landed, in, fields, belonging, to,...","[O, O, O, O, O, O, O, O, O, O, O]"
47957,47958,"[They, say, not, all, of, the, rockets, explod...","[O, O, O, O, O, O, O, O, O, O, O]"


In [1]:
from dataset import NERDataset

hf = NERDataset(dataset_path="dataset/ner_dataset.csv")
hf = hf.prepare_hf_dataset()

In [22]:
hf['train']

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 26976
})

In [23]:
hf['train'].features['tokens'].feature

Value(dtype='string', id=None)

In [18]:
import datasets

In [37]:
from datasets import ClassLabel, Value

new_features = hf['train'].features.copy()
new_features["ner_tags"] = ClassLabel(names=["negative", "positive"])
# new_features["id"] = Value("int64")
# new_features["tokens"] = Value("list")
hf['train'] = hf['train'].cast(new_features)
hf['train'].features

Casting the dataset:   0%|          | 0/26976 [00:00<?, ? examples/s]

TypeError: Couldn't cast array of type
list<item: string>
to
int64

In [27]:
features = {
    'id': datasets.Value(dtype='int64', id=None),
    'tokens': datasets.Sequence(datasets.Value(dtype='string', id=None), length=-1, id=None),
    'ner_tags': datasets.Sequence(datasets.ClassLabel(names=['PERSON', 'LOCATION', 'ORGANIZATION']), length=-1, id=None),
}

hf = hf.cast(features)

AttributeError: 'dict' object has no attribute 'arrow_schema'

In [29]:
datasets.__version__

'2.14.5'